## Reset the cluster

In [1]:
kind delete cluster

Deleting cluster "kind" ...
Deleted nodes: ["kind-control-plane"]


In [2]:
kind create cluster

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.30.0) 🖼
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Thanks for using kind! 😊


## Install the dashboard

In [3]:
helm repo add kubernetes-dashboard https://kubernetes.github.io/dashboard/

"kubernetes-dashboard" already exists with the same configuration, skipping


In [4]:
helm repo list

NAME                	URL                                    
kubernetes-dashboard	https://kubernetes.github.io/dashboard/


In [5]:
helm install dashboard kubernetes-dashboard/kubernetes-dashboard -n monitoring --create-namespace

NAME: dashboard
LAST DEPLOYED: Sun Jul 21 11:28:08 2024
NAMESPACE: monitoring
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
*************************************************************************************************
*** PLEASE BE PATIENT: Kubernetes Dashboard may need a few minutes to get up and become ready ***
*************************************************************************************************

Congratulations! You have just installed Kubernetes Dashboard in your cluster.

To access Dashboard run:
  kubectl -n kubernetes-dashboard port-forward svc/kubernetes-dashboard-kong-proxy 8443:443

NOTE: In case port-forward command does not work, make sure that kong service name is correct.
      Check the services in Kubernetes Dashboard namespace using:
        kubectl -n monitoring get svc

Dashboard will be available at:
  https://localhost:8443


In [6]:
helm list -n monitoring

NAME     	NAMESPACE 	REVISION	UPDATED                                	STATUS  	CHART                     	APP VERSION
dashboard	monitoring	1       	2024-07-21 11:28:08.921614551 +0100 BST	deployed	kubernetes-dashboard-7.5.0	           


In [7]:
sleep 10
kubectl get pods -n monitoring

NAME                                                              READY   STATUS              RESTARTS   AGE
dashboard-kong-68687498db-6z74f                                   0/1     Init:0/1            0          2s
dashboard-kubernetes-dashboard-api-55874568c8-hwbv2               0/1     ContainerCreating   0          2s
dashboard-kubernetes-dashboard-auth-64f65c7d7-v54hl               0/1     ContainerCreating   0          2s
dashboard-kubernetes-dashboard-metrics-scraper-689c7b5b74-kmlfk   0/1     ContainerCreating   0          2s
dashboard-kubernetes-dashboard-web-548b8b8859-cvhhs               0/1     ContainerCreating   0          2s


In [8]:
kubectl get deployment -n monitoring

NAME                                             READY   UP-TO-DATE   AVAILABLE   AGE
dashboard-kong                                   0/1     1            0           11s
dashboard-kubernetes-dashboard-api               0/1     1            0           11s
dashboard-kubernetes-dashboard-auth              0/1     1            0           11s
dashboard-kubernetes-dashboard-metrics-scraper   0/1     1            0           11s
dashboard-kubernetes-dashboard-web               0/1     1            0           11s


In [9]:
kubectl get sa -n monitoring

NAME                                             SECRETS   AGE
dashboard-kong                                   0         11s
dashboard-kubernetes-dashboard-api               0         11s
dashboard-kubernetes-dashboard-metrics-scraper   0         11s
dashboard-kubernetes-dashboard-web               0         11s
default                                          0         3s


## Access the dashboard

In [10]:
kubectl -n monitoring create clusterrolebinding dashboard-kubernetes-dashboard --clusterrole=cluster-admin --serviceaccount=monitoring:default

clusterrolebinding.rbac.authorization.k8s.io/dashboard-kubernetes-dashboard created


In [11]:
# kubectl -n monitoring port-forward svc/dashboard-kong-proxy 8443:443

In [12]:
kubectl -n monitoring create token default

eyJhbGciOiJSUzI1NiIsImtpZCI6IkdSS0ZBNk9PZlRSS0d5NFpjaUhHay02QlZ4VVhSd0gyS0w0YnRORFdaSFUifQ.eyJhdWQiOlsiaHR0cHM6Ly9rdWJlcm5ldGVzLmRlZmF1bHQuc3ZjLmNsdXN0ZXIubG9jYWwiXSwiZXhwIjoxNzIxNTYxMzAxLCJpYXQiOjE3MjE1NTc3MDEsImlzcyI6Imh0dHBzOi8va3ViZXJuZXRlcy5kZWZhdWx0LnN2Yy5jbHVzdGVyLmxvY2FsIiwianRpIjoiNDViMGQxYzctZjMzYy00MjgxLWE3ODctYmI4NzdiNTBkZmM5Iiwia3ViZXJuZXRlcy5pbyI6eyJuYW1lc3BhY2UiOiJtb25pdG9yaW5nIiwic2VydmljZWFjY291bnQiOnsibmFtZSI6ImRlZmF1bHQiLCJ1aWQiOiI1OGEzMDEzNi1jZDIyLTQ4ZjgtYTZmYS1iMzMzNjJhM2Q2OWUifX0sIm5iZiI6MTcyMTU1NzcwMSwic3ViIjoic3lzdGVtOnNlcnZpY2VhY2NvdW50Om1vbml0b3Jpbmc6ZGVmYXVsdCJ9.XLoicwlk_BdIbmPEieHaGfFXHsaqLDRSvLl6xm68cIft3hIF9xpt8xm_Q5qnSka3O3UyYZpdT_HHgN19JsyMSuot2NHf3I7EZ8UL85qYZUfEjjihUknhpXkDxeZbnNpx8uYFKT8Gf8L7Mc0xsTV4RoMCuWpDWGjXfrIioJljBvbeYraHsIyHeo3zDJGfZ1767c1Uf-lF1luXm5b009pddoJu3PAL6I3oyp_9RthUHWo-jkVu_9bzK0YOMP-CrNTPe5gT2JAaU3EfhFJzUOKWK3KAsDAENFSGHAH8avN4jAW3cr6RrmmjeBh-g5nbHr2oynqKIEsSLsP4Rj6y09tSlw


## Upgrade the dashboard using custom values

`dashboard.yaml` enables metrics on the dashboard (e.g. CPU and memory usage)

In [13]:
# USING --wait VALIDATES THE NEW RELEASE, ALWAYS USE THIS
helm upgrade dashboard kubernetes-dashboard/kubernetes-dashboard -n monitoring --wait --timeout 60s --values dashboard.yaml

Release "dashboard" has been upgraded. Happy Helming!
NAME: dashboard
LAST DEPLOYED: Sun Jul 21 11:28:22 2024
NAMESPACE: monitoring
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
*************************************************************************************************
*** PLEASE BE PATIENT: Kubernetes Dashboard may need a few minutes to get up and become ready ***
*************************************************************************************************

Congratulations! You have just installed Kubernetes Dashboard in your cluster.

To access Dashboard run:
  kubectl -n kubernetes-dashboard port-forward svc/kubernetes-dashboard-kong-proxy 8443:443

NOTE: In case port-forward command does not work, make sure that kong service name is correct.
      Check the services in Kubernetes Dashboard namespace using:
        kubectl -n monitoring get svc

Dashboard will be available at:
  https://localhost:8443


In [14]:
helm list -n monitoring

NAME     	NAMESPACE 	REVISION	UPDATED                                	STATUS  	CHART                     	APP VERSION
dashboard	monitoring	2       	2024-07-21 11:28:22.239430246 +0100 BST	deployed	kubernetes-dashboard-7.5.0	           


In [15]:
helm history dashboard -n monitoring

REVISION	UPDATED                 	STATUS    	CHART                     	APP VERSION	DESCRIPTION     
1       	Sun Jul 21 11:28:08 2024	superseded	kubernetes-dashboard-7.5.0	           	Install complete
2       	Sun Jul 21 11:28:22 2024	deployed  	kubernetes-dashboard-7.5.0	           	Upgrade complete


## Deploy and rollback a bad configuration

In [16]:
# USING --wait VALIDATES THE NEW RELEASE, ALWAYS USE THIS
helm upgrade dashboard kubernetes-dashboard/kubernetes-dashboard -n monitoring --wait --timeout 60s --values error.yaml

Error: UPGRADE FAILED: context deadline exceeded


: 1

In [17]:
helm list -n monitoring

NAME     	NAMESPACE 	REVISION	UPDATED                                	STATUS	CHART                     	APP VERSION
dashboard	monitoring	3       	2024-07-21 11:29:07.516189652 +0100 BST	failed	kubernetes-dashboard-7.5.0	           


In [18]:
helm history dashboard -n monitoring

REVISION	UPDATED                 	STATUS    	CHART                     	APP VERSION	DESCRIPTION                                          
1       	Sun Jul 21 11:28:08 2024	superseded	kubernetes-dashboard-7.5.0	           	Install complete                                     
2       	Sun Jul 21 11:28:22 2024	deployed  	kubernetes-dashboard-7.5.0	           	Upgrade complete                                     
3       	Sun Jul 21 11:29:07 2024	failed    	kubernetes-dashboard-7.5.0	           	Upgrade "dashboard" failed: context deadline exceeded


In [19]:
helm rollback dashboard 2 -n monitoring --wait --timeout 60s

Rollback was a success! Happy Helming!


In [20]:
helm history dashboard -n monitoring

REVISION	UPDATED                 	STATUS    	CHART                     	APP VERSION	DESCRIPTION                                          
1       	Sun Jul 21 11:28:08 2024	superseded	kubernetes-dashboard-7.5.0	           	Install complete                                     
2       	Sun Jul 21 11:28:22 2024	superseded	kubernetes-dashboard-7.5.0	           	Upgrade complete                                     
3       	Sun Jul 21 11:29:07 2024	failed    	kubernetes-dashboard-7.5.0	           	Upgrade "dashboard" failed: context deadline exceeded
4       	Sun Jul 21 11:31:12 2024	deployed  	kubernetes-dashboard-7.5.0	           	Rollback to 2                                        


## Uninstall and restore the dashboard

In [21]:
helm uninstall dashboard -n monitoring --keep-history

release "dashboard" uninstalled


In [22]:
sleep 10
kubectl get all -n monitoring

NAME                                  READY   STATUS        RESTARTS   AGE
pod/dashboard-kong-68687498db-6z74f   1/1     Terminating   0          3m5s


In [23]:
helm rollback dashboard 2 -n monitoring --wait --timeout 60s

Rollback was a success! Happy Helming!


In [24]:
sleep 10
kubectl get all -n monitoring

NAME                                                                  READY   STATUS    RESTARTS   AGE
pod/dashboard-kong-68687498db-9lqc8                                   1/1     Running   0          32s
pod/dashboard-kubernetes-dashboard-api-c44b5f55b-l88m4                1/1     Running   0          32s
pod/dashboard-kubernetes-dashboard-auth-6569f9b59-c7vpl               1/1     Running   0          32s
pod/dashboard-kubernetes-dashboard-metrics-scraper-689c7b5b74-jd4w8   1/1     Running   0          32s
pod/dashboard-kubernetes-dashboard-web-548b8b8859-fvmrb               1/1     Running   0          32s
pod/dashboard-metrics-server-c76565f8c-2f7gv                          1/1     Running   0          32s

NAME                                                     TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)                         AGE
service/dashboard-kong-manager                           NodePort    10.96.97.131    <none>        8002:30278/TCP,8445:31657/TCP   32s
service/

In [25]:
helm history dashboard -n monitoring

REVISION	UPDATED                 	STATUS     	CHART                     	APP VERSION	DESCRIPTION                                          
1       	Sun Jul 21 11:28:08 2024	superseded 	kubernetes-dashboard-7.5.0	           	Install complete                                     
2       	Sun Jul 21 11:28:22 2024	superseded 	kubernetes-dashboard-7.5.0	           	Upgrade complete                                     
3       	Sun Jul 21 11:29:07 2024	failed     	kubernetes-dashboard-7.5.0	           	Upgrade "dashboard" failed: context deadline exceeded
4       	Sun Jul 21 11:31:12 2024	uninstalled	kubernetes-dashboard-7.5.0	           	Uninstallation complete                              
5       	Sun Jul 21 11:31:24 2024	deployed   	kubernetes-dashboard-7.5.0	           	Rollback to 2                                        
